In [ ]:
#load libraries 
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point,Polygon,LineString

import googlemaps
from datetime import datetime
import time

import matplotlib.pyplot as plt
%matplotlib inline

# City grid

In [ ]:
#build a dict of cities with the polygon as it appears on Nominatim so it can be downloaded from OSM with osmnx
#and an administative center 
cities = {'buenos_aires':{'nominatim_name':'Buenos Aires,Argentina',
                    'admin_center':(-34.601078, -58.374465)},
           'mexico':{'nominatim_name':{'state':'Ciudad de México', 'country':'México'},
                    'admin_center':(19.4326009,-99.1333416)},
           'san_pablo':{'nominatim_name':{'city':'San Pablo','state':'San Pablo', 'country':'Brazil'},
                       'admin_center':(-23.5506507,-46.6333824)},
            'amsterdam':{'nominatim_name':'Ámsterdam, Holanda Septentrional, Países Bajos',
                        'admin_center':(52.3727598, 4.8936041)},
            'nyc':{'nominatim_name':'New York City, USA',
                  'admin_center':(40.712701, -74.006129)}
           }

In [ ]:
def get_city_grid(city_name,cell_height = 1000,cell_width = 1000, intersection = True):
    '''
    This function takes a city name from a the cities dict,
    a cell width and height
    an wether we want to intersect the cells with the city boudnary
    and returns a shapefile with a polygon grid for the city in question
    '''
    city = ox.gdf_from_place(city_name)
    city = city.to_crs(epsg = 3857) 
    xmin,ymin,xmax,ymax =  city.geometry.item().bounds

    rows = int(np.ceil((ymax-ymin) /  cell_height))
    columns = int(np.ceil((xmax-xmin) / cell_width))

    XleftOrigin = xmin
    XrightOrigin = xmin + cell_width
    YtopOrigin = ymax
    YbottomOrigin = ymax - cell_height
    #create empy list for polygons 
    polygons = []
    #first go for every columns
    for i in range(columns):
        Ytop = YtopOrigin
        Ybottom =YbottomOrigin
        #within every column, create cells
        for j in range(rows):
            #create a polygon from heach node of the cell
            polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
            #update point coordinates
            Ytop = Ytop - cell_height
            Ybottom = Ybottom - cell_height
        XleftOrigin = XleftOrigin + cell_width
        XrightOrigin = XrightOrigin + cell_width

    #build a geodataframe from polygons
    grid = gpd.GeoDataFrame({'geometry':polygons})
    grid.crs = {'init':'epsg:3857','no_defs': True}
    if intersection:
        
        grid = gpd.overlay(grid, city, how='intersection')
        grid.crs = {'init':'epsg:3857','no_defs': True}
         
    grid = grid.to_crs(epsg = 4326) 
    grid = grid.reset_index().reindex(columns=['index','geometry'])
    
    return grid

def query_gmaps(gmaps=gmaps,O, D, mode,depart):
    '''
    This functions takes 
    a gmaps instance
    coordinates for Origin and Destination 
    O = (origin.y,origin.x)
    D = (destination.y,destination.x)
    a mode: bicycling o driving
    and a departure time
    and returns a google distance matrix api
    '''
    out = gmaps.distance_matrix(
        origins = O,
        destinations = D,
        language=None, avoid=None,
        mode=mode,
        units = 'metric',
        departure_time = depart,
        arrival_time=None, transit_mode=None,
        transit_routing_preference=None,
        traffic_model="best_guess",
        region = None)
    return out


In [ ]:
city = 'buenos_aires'
nominatim_name = cities[city]['nominatim_name']
admin_center = cities[city]['admin_center']
cell_size = 500

In [ ]:
city_grid = get_city_grid(city_name = nominatim_name,cell_height = cell_size,cell_width = cell_size, intersection = True)

In [ ]:
centroids = city_grid.centroid

In [ ]:
#https://developers.google.com/maps/documentation/distance-matrix/usage-and-billing#distance-matrix-advanced
#cost of transaction in USD dollars
u_cost_driving = 0.01
u_cost_bike = 0.005
total_cost = len(centroids)* u_cost_driving + len(centroids)* u_cost_bike
print (city,total_cost)

# Travel time by mode

In [ ]:
#intert a google distance matrix API here
api = 'xxxxx'
gmaps = googlemaps.Client(key=api)
depart = int(time.mktime(datetime(2019, 4, 17, 9, 0, 0).timetuple()))

In [ ]:
city_grid['bike_query'] = centroids.apply(lambda origin: query_gmaps(O = (origin.y,origin.x), D = admin_center, mode = 'bicycling',depart = depart))

In [ ]:
city_grid['drive_query'] = centroids.apply(lambda origin: query_gmaps(O = (origin.y,origin.x), D = admin_center, mode = 'driving',depart = depart))

In [ ]:
#save in geojson for full preservation of API result
city_grid.to_file(city+'_'+str(cell_size)+'.geojson',driver = 'GeoJSON')

In [ ]:
city_grid.to_file(city+'_'+str(cell_size))